In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
import re
import pandas as pd
from IPython.display import display, HTML

In [2]:
options = Options()
options.add_experimental_option('detach',True) # to open window after work is done
 
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()),options=options)

driver.maximize_window() # for full screen
driver.get("https://edusmartz.ssuet.edu.pk/StudentPortal/Login") # link to visit
driver.implicitly_wait(30)

In [3]:
rollno = driver.find_element("xpath","//input[@id='txtRegistrationNo_cs']")
rollno.clear()
rollno.send_keys("2020F-BCS-096")

password = driver.find_element("xpath","//input[@id='txtPassword_m6cs']")
password.clear()
password.send_keys("9E5D0F3C82CA")

signinbtn = driver.find_element("xpath","//input[@id='btnlgn']")
signinbtn.click()

driver.implicitly_wait(30)

In [4]:
sidebar = driver.find_elements("xpath","//li[@class='menu-item menu-item-submenu']")
sidebar[0].click()

In [5]:
attendance = driver.find_element("xpath","//li[@class='menu-item'][.//span[text()[contains(.,'Attendance')]]]")
attendance.get_attribute("innerHTML")
attendance.click()

In [6]:
table = driver.find_element("xpath","//table[@id='ctl00_ContentPlaceHolder1_TgridAttedance_ctl00']")

In [7]:
text = table.get_attribute("innerHTML")
driver.quit()

In [8]:
start_text = '<th scope="col" class="rgHeader">View'
end_text = "</th>"  # Ending text
split_text = text.split(start_text, 1)
text = split_text[0] + split_text[1].split(end_text, 1)[1]

start_text = '<colgroup>'
end_text = "</colgroup>"  # Ending text
split_text = text.split(start_text, 1)
text = split_text[0] + split_text[1].split(end_text, 1)[1]


start_text = "<a id="  # Starting text
end_text = "</a>"  # Ending text

pattern = re.escape(start_text) + "(.*?)" + re.escape(end_text)
attendance_table = re.sub(pattern, "", text)
attendance_table = "<table>" + attendance_table + "</table>"

In [9]:
# Read HTML table string into a list of dataframes
tables = pd.read_html(attendance_table)

# Assuming the desired table is the first one
table_df = tables[0]

# Save the dataframe as an Excel file
table_df.to_excel('attendance.xlsx', index=False)

In [10]:
display(HTML(attendance_table))